In [1]:
import sys
sys.path.insert(0,"")

from StreetGraph import StreetGraph
from gym_graphenv.envs.Graphworld import GraphEnv


#from gym_example.envs.Gridworld_Gym import Gridworld_v1


#graph_meinheim=StreetGraph('meinheim')
#graph_meinheim_trips = graph_meinheim.trips

In [2]:
import pandas as pd
df = pd.read_csv("Provider.csv")
df[df['id']==1]['basic_cost']
provider_id = df['id'].sample(n=1)
print(provider_id)
print(provider_id.iloc[0])

0    1
Name: id, dtype: int64
1


In [2]:
env=GraphEnv()

{'y': 49.0160883, 'x': 10.7898611, 'street_count': 3}
{'y': 49.0137365, 'x': 10.7909045, 'street_count': 3}
{'y': 49.0110678, 'x': 10.8036815, 'street_count': 3}
{'y': 49.0190163, 'x': 10.8012552, 'street_count': 3}
{'y': 49.0256019, 'x': 10.8085108, 'street_count': 4}
{'y': 49.0262161, 'x': 10.8095749, 'street_count': 3}
{'y': 49.0266088, 'x': 10.8103372, 'street_count': 3}
{'y': 49.0271074, 'x': 10.8110651, 'street_count': 3}
{'y': 49.0280664, 'x': 10.8105361, 'street_count': 3}
{'y': 49.0283535, 'x': 10.8105211, 'street_count': 3}
{'y': 49.0304953, 'x': 10.8169647, 'street_count': 3}
{'y': 49.0303305, 'x': 10.8157946, 'street_count': 3}
{'y': 49.0302912, 'x': 10.8155929, 'street_count': 3}
{'y': 49.0301588, 'x': 10.8156699, 'street_count': 3}
{'y': 49.0295097, 'x': 10.8133491, 'street_count': 3}
{'y': 49.0291017, 'x': 10.8127015, 'street_count': 3}
{'y': 49.0288544, 'x': 10.8118226, 'street_count': 3}
{'y': 49.0283535, 'x': 10.8105211, 'street_count': 3}
{'y': 49.0280664, 'x': 10.81

In [ ]:
env.availableActions()

In [ ]:
env.step(action=2)

In [ ]:
env.graph.get_nodeid_by_index(env.position)

In [ ]:
graph_meinheim=StreetGraph('meinheim')
graph_meinheim_trips = StreetGraph('meinheim').trips
print(graph_meinheim_trips)

In [ ]:
#without ppo trainer
def run_one_episode (env):
    env.reset()
    sum_reward = 0
    for i in range(30):
        print(env.action_space)
        action = env.action_space.sample()
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("sum_reward: ",sum_reward, " time: ",env.time,  "deadline time: ", env.deadline,"pickup time: ", env.pickup_time )
            break

        print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

env=GraphEnv()
for i in range(1):
    run_one_episode (env)

In [ ]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym

import ray
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

In [ ]:
ray.init()

In [ ]:
trainer_config = DEFAULT_CONFIG.copy()
trainer_config['num_workers'] = 1
trainer_config["train_batch_size"] = 400
trainer_config["sgd_minibatch_size"] = 64
trainer_config["num_sgd_iter"] = 10
trainer_config["framework"] = "torch"

In [ ]:
trainer = PPOTrainer(trainer_config,GraphEnv )

In [ ]:
checkpoint_root = "tmp/ppo/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

In [ ]:
trainer.train()

In [ ]:
results = []
episode_data = []
episode_json = []
n_iter=5

for n in range(n_iter):
    result = trainer.train()
    results.append(result)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

In [ ]:
trainer.restore(file_name)
env = gym.make("graphworld-v0")
state = env.reset()

In [ ]:
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = trainer.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == 1:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

In [ ]:
ray.shutdown()